# Inferência de trajetória e ordenação pseudotemporal

## Instale as bibliotecas necessárias

In [ ]:
# Baixe um script do GitHub que configura a instalação dos pacotes # pra o R do gerenciador de pacotes do sistema (apt)
download.file("https://github.com/eddelbuettel/r2u/raw/master/inst/scripts/add_cranapt_jammy.sh",
              "add_cranapt_jammy.sh")

# Conceda a permissão de execução ao script baixado
Sys.chmod("add_cranapt_jammy.sh", "0755")

# Execute o script para estabelecer a instalação do pacote R via apt
system("./add_cranapt_jammy.sh")

# Habilite o bspm (Bridge to System Package Manager), que permite instalar os pacotes do R para o gerenciador de pacotes do sistema
bspm::enable()

# Vesabilite a verificação de versão do bspm para evitar problemas de compatibilidade
options(bspm.version.check=FALSE)

# Remova o script após a execução para manter o ambiente limpo
system("rm add_cranapt_jammy.sh")

## Vamos criar uma função R para realizar a chamada do sistema

In [ ]:
# Defina uma função para executar o comando shell e salve sua saída
shell_call <- function(command, ...) {
  # Execute o comando no system shell e salve a saída
  result <- system(command, intern = TRUE, ...)
  
  # Imprima a saída em um formato legível
  cat(paste0(result, collapse = "\n"))
}

## Instale os pacotes necessários

In [ ]:
# Instale o pacote R.utils, que fornece utilidades adicionais para os próximos passos
install.packages("R.utils")

# Instale versões específicas do Seurat Wrappers e Seurat Data do GitHub
remotes::install_github('satijalab/seurat-wrappers@d28512f804d5fe05e6d68900ca9221020d52cf1d', upgrade=F)
remotes::install_github('satijalab/seurat-data')

# Verifique se o BiocManager está instalado, caso contrário, instale-o para gerenciar os pacotes do Bioconductor
if (!require("BiocManager", quietly = TRUE))
    install.packages("BiocManager", quiet = T)

# Instale os pacotes adicionais necessários
install.packages("harmony")  # Harmony para correções de lote
BiocManager::install("clusterProfiler", update = T, ask=F, force=T) # Para análises de enriquecimentos funcionais
BiocManager::install("destiny", update = F) # Para mapas de difusão para dados de células únicas
remotes::install_github('cole-trapnell-lab/monocle3') # Instale Monocle3 para inferências de trajetória

# Opicioal: Instale uma versão específica do pacote Matrix (comentado)
# install.packages("https://cran.r-project.org/src/contrib/Archive/Matrix/Matrix_1.5-3.tar.gz", repos=NULL, type="source")


# Introdução

Células continuamente transitam entre diferentes estados funcionais através do desenvolvimento e da vida. Durante essa transição, a expressão gênica muda dinamicamente com a ativação de alguns genes, enquanto outros são silenciados. O sequenciamento de RNA de células únicas (scRNA-seq), permite aos pesquisadores identificar essa mudança dinâmica em alta resolução.

Ferramentas computacionais como o Monocle3 usa dados de scRNA-seq para reconstruir trajetórias celulares, nos ajudando a compreender como células progridem através de diferentes estados ao longo do tempo. Essa abordagem é particularmente útil em estudos de diferenciação, progressão de doenças e reprogramação celular.

Nesse tutorial, nós vamos aprender como inferir trajetórias celulares e estimar o pseudotempo - uma medida da progressão relativa das células ao longo do desenvolimento - usando o Monocle3. Ao analisar dados de células únicas, nós podemos mapear como células evoluem através de diferentes estados funcionais e identificar genes chave que direcionam essas transições.

Esse tutorial é inspirado e baseia-se em guias de estudos anteriores que demonstram o poder da inferência de trajetória na biologia de estudos de células únicas.
- [Turorial original do Monocle3](https://cole-trapnell-lab.github.io/monocle3/docs/trajectories/)
- [Tutorial combinando o Seurat e o Monocle3 pelo Stuart Lab](https://stuartlab.org/signac/articles/monocle.html)
- [Tutorial combinando Seurat and Monocle3 pela Mahima Bose](https://rpubs.com/mahima_bose/Seurat_and_Monocle3_p)

![Monocle3](https://cole-trapnell-lab.github.io/monocle3/images/manual_images/embryo_pr_graph_by_pseudotime.png)

In [ ]:
# Carregue as bibliotecas necessárias para as análises de sequenciamento de células únicas
library(monocle3)      # Inferência de trajetórias
library(Seurat)        # Estrutura de análises de células únicas
library(SeuratData)    # Conjuntos de dados de células únicas pré-processados
library(SeuratWrappers) # Funcionalidades adicionais do Seurat
library(patchwork)     # Composições de plotagens
library(harmony)       # Correção de lotes
library(ggplot2)       # Visualização da dados

## Carregando dados

Aqui, iremos carregar nosso conjunto de dados de interesse para integrar múltiplas amostras de scRNA-seq.

Esse tutorial irá demonstrar como alinhar duas amostras células mononucleares de sangue periférico (PBMCs) do [Kang et al, 2017](https://www.nature.com/articles/nbt.4042). Nesse experimento, os PBMCs foram divididos em grupo controle e grupo estimulado, que foi tratado com interferon-beta. Esse estímulo levou mudanças celulares tipo-específicas na expressão gênica. Como resultado, quando analisamos os dados, as células tendem a se agrupar não apenas por sua identidade biológica, mas também pela condição estimulada. Isso introduz um desafio para as nossas análises, pois diferenças nos padrões de expressão podem obscurecer as semelhanças subjacentes entre os mesmos tipos celulares em ambos os grupos.

Para integrar esses conjuntos de dados, nós iremos corrigir os efeitos de lote e as variações atribuídas pela condição específica, permitindo maior acurácia nas comparações das características biológicas compartilhadas entre ambos os grupos.

In [ ]:
# Baixe e instale o conjunto de dados "ifnb", que contém os dados de scRNA-seq.
InstallData("ifnb")

In [ ]:
# Carregue o conjunto de dados previamente instalado "ifnb"
LoadData("ifnb")

In [ ]:
# Armazene o conjunto de dados carregado em uma nova variável chamada 'testdata'
# Isso permite modifica o conjunto de dados enquanto deixamos os dados originais intactos
testdata <- ifnb

In [ ]:
# Certifique-seq que o objeto Seurat está atualizado para o formato mais recente
testdata <- UpdateSeuratObject(object = testdata)

# Obtenha uma visão geral do conjunto de dados usando o dplyr::glimpse()
testdata %>% dplyr::glimpse()

## Processamento de dados

Nós vamos realizar o típico processamento de dados, integração, correção de lote e clusterização antes de rodar o Monocle3.

In [ ]:
# Aqui está o passo-a-passo da estapa de processamento
testdata <- Seurat::NormalizeData(testdata, verbose = FALSE) %>%  # Normalize 
            FindVariableFeatures(selection.method = "vst", nfeatures = 2000) %>%  # Identificar os 2000 genes mais variáveis
            ScaleData(verbose = FALSE) %>%  # Padronizar e centralizar os dados
            RunPCA(npcs = 30, verbose = FALSE) %>%  # Executar Análises de Componentes Principais (PCA) com 30 componentes
            RunHarmony("stim", plot_convergence = FALSE) %>%  # Correção de lote usando o Harmony
            RunUMAP(reduction = "harmony", dims = 1:30) %>%  # Executar o agrupamento por UMAP usando os dados corrigidos pelo Harmony
            FindNeighbors(reduction = "harmony", dims = 1:30) %>%  # Compute os vizinhos mais próximos para agrupamento 
            FindClusters(resolution = 0.5)  # Agrupe as células usando o algorítmo Louvain

# O argumento 'verbose = FALSE' suspende as mensagens de saída para manter o console limpo

É assim que o UMAP se parece:

In [ ]:
# Crie um UMAP com os rótulos dos agrupamentos baseados no 'seurat_annotations'
scPlot <- DimPlot(testdata, label = TRUE, group.by = 'seurat_annotations')
# Para exibir o gráfico
scPlot

# Opicional: Salve o gráfico como uma imagem (comentado)
# ggsave("01-DimPlot.png", plot = scPlot, bg = "white")

## Usando o Monocle3

Para analisar a trajetória celular usando o Monocle3, primeiramente nós precisamos converter nosso objeto Seurat para o formato que o Monocle3 consegue processar. Isso pode ser feito usando a função `as.cell_data_set()` do pacote SeuratWrappers. Essa função transforma o objeto Seurat em um objeto CellDataSet, que serve como arquivo de entrada para o algoritmo de inferência de trajetória do Monocole3. Uma vez convertido, esse objeto pode ser utilizado para construção trajetórias de desenvolvimento, inferências de pseudotempo e analisar transições de estágios celulares.

In [ ]:
# Convert Seurat object into a Monocle3-compatible cell_data_set (cds)
cds <- as.cell_data_set(testdata)

# Store gene names as metadata in the cell dataset
fData(cds)$gene_short_name <- rownames(fData(cds))

In [ ]:
# Get an overview of the structure of the cell dataset (cds)
cds %>% dplyr::glimpse()

In [ ]:
cds <- cluster_cells(cds = cds,  # Perform clustering on the cell dataset
                     reduction_method = "UMAP",  # Use UMAP for dimensionality reduction
                     cluster_method = 'louvain') %>%  # Apply Louvain algorithm for clustering
       learn_graph(use_partition = T)  # Learn the cellular trajectory graph

In [ ]:
# Generate a UMAP plot showing cell clusters along with trajectory landmarks  
scPlot <- plot_cells(cds, 
                     color_cells_by = "cluster",  # Color cells by cluster assignment  
                     label_groups_by_cluster = FALSE,  # Do not label clusters  
                     label_branch_points = TRUE,  # Label branch points in the trajectory  
                     label_roots = TRUE,  # Label the root cells in the trajectory  
                     label_leaves = TRUE,  # Label the leaf nodes in the trajectory  
                     group_label_size = 5)  # Set the font size for labels  

# Display the plot  
scPlot  

# Optional: Save the plot as an image  
# ggsave("02-plot_cells.png", plot = scPlot, bg = "white", width = 9, height = 9, dpi = 600)

In [ ]:
# Generate a UMAP plot showing cell clusters without trajectory landmarks  
scPlot <- plot_cells(cds, 
                     color_cells_by = "cluster",  # Color cells by cluster assignment  
                     label_groups_by_cluster = FALSE,  # Do not label clusters  
                     label_branch_points = FALSE,  # Do not label branch points  
                     label_roots = FALSE,  # Do not label root cells  
                     label_leaves = FALSE,  # Do not label leaf nodes  
                     group_label_size = 5)  # Set the font size for labels  

# Display the plot  
scPlot  

# Optional: Save the plot as an image  
# ggsave("03-plot_cells.png", plot = scPlot, bg = "white", width = 9, height = 9, dpi = 600)

In [ ]:
# Set plot size (optional)  
# options(repr.plot.height = 9, repr.plot.width = 16)

# Create a UMAP plot with cluster annotations from Seurat  
gumap <- DimPlot(testdata, label = TRUE, group.by = 'seurat_annotations')

# Create a Monocle3 plot showing clusters without trajectory landmarks  
gcluster <- plot_cells(cds, 
                       color_cells_by = "cluster",  
                       label_groups_by_cluster = FALSE,  
                       label_branch_points = FALSE,  
                       label_roots = FALSE,  
                       label_leaves = FALSE,  
                       group_label_size = 5)

# Combine both plots into a single figure  
scPlot <- gumap + gcluster + theme(aspect.ratio = 1)

# Display the combined plot  
scPlot  

# Optional: Save the plot as an image  
# ggsave("04-DimPlot-plot_cells.png", plot = scPlot, bg = "white", width = 18, height = 9, dpi = 600)

In [ ]:
# Assign a pseudotemporal order to cells using specific clusters as root cells  
cds <- order_cells(cds, 
                   reduction_method = "UMAP",  # Use UMAP for trajectory inference  
                   root_cells = colnames(cds[, clusters(cds) %in% c(3, 15, 9, 22)]))  # Specify root clusters  

In [ ]:
# Set plot size (optional)  
options(repr.plot.height = 7, repr.plot.width = 7)

# Generate a UMAP plot with cells colored by pseudotime  
scPlot1 <- plot_cells(cds, 
                      color_cells_by = "pseudotime",  # Color cells based on their pseudotime  
                      label_groups_by_cluster = FALSE,  
                      label_branch_points = FALSE,  
                      label_roots = FALSE,  
                      label_leaves = FALSE,  
                      group_label_size = 5)

# Display the plot  
scPlot1  

# Save the plot as an image  
ggsave("05-plot_cells.png", plot = scPlot1, bg = "white", width = 9, height = 9, dpi = 600)

In [ ]:
# Set plot size (optional)  
# options(repr.plot.height=6, repr.plot.width=16)

# Combine previous plots (Seurat UMAP, Monocle3 clusters, and pseudotime)  
scPlot <- gumap + gcluster + scPlot1

# Display the combined figure  
scPlot  

# Save the combined plot as an image  
ggsave("06-Multiple_plots.png", plot = scPlot, bg = "white", width = 27, height = 9, dpi = 600)

In [ ]:
# Set plot size (optional)  
# options(repr.plot.height=7, repr.plot.width=7)

# Extract pseudotime values from Monocle3  
cds$monocle3_pseudotime <- pseudotime(cds)

# Convert cell metadata into a dataframe  
data.pseudo <- as.data.frame(colData(cds))

# Generate a boxplot showing pseudotime distribution across cell types  
scPlot <- ggplot(data.pseudo, aes(monocle3_pseudotime, 
                                  reorder(seurat_annotations, monocle3_pseudotime),  # Order by pseudotime  
                                  fill = seurat_annotations)) +  # Color by cell type  
          geom_boxplot()  # Create boxplot  

# Display the boxplot  
scPlot  

# Optional: Save the plot as an image  
# ggsave("07-boxplot.png", plot = scPlot, bg = "white")

In [ ]:
# Extract expression data for selected genes (CD44 and CXCL2)  
cds_subset <- cds[c('CD44', 'CXCL2'), ]

In [ ]:
# Generate a plot showing expression of selected genes across pseudotime  
scPlot <- plot_genes_in_pseudotime(cds_subset)

# Display the plot  
scPlot  

# Optional: Save the plot as an image  
# ggsave("08-genes_in_pseudotime.png", plot = scPlot, bg = "white")

In [ ]:
# The code below identifies genes that change their expression over pseudotime.  
# However, running this can be time-consuming.  

# cds_pr_test_res <- graph_test(cds, neighbor_graph="principal_graph", cores=4)  # Perform differential expression analysis  
# pr_deg_ids <- row.names(subset(cds_pr_test_res, q_value < 0.05))  # Select significant genes based on q-value 